<a href="https://colab.research.google.com/github/JarredParrettDickinson/AG_DATA_ANALYSIS/blob/master/STATE_ECONOMIC_DATA_SUMMARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Welcome to my widget to present State Economic Agricultre Data Geographically.
## The following steps will direct you to operate the notebook if you are not familiar. 


1.   On the header of the Google CoLab, select `Runtime` -> `Run All`
2.   Scroll down to `Using the interactive portion` below to plot on a map 






In [0]:
import pandas as pd
import plotly
import plotly.express as px
from gzip import GzipFile
import requests
from urllib.request import Request, urlopen
import urllib.request as ur
import numpy as np
import requests
import json
from pandas.io.json import json_normalize 
import io
from ipywidgets import widgets
import math
from google.colab import drive
import urllib
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
from IPython.display import display
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import datetime as dt
from urllib.request import urlopen

In [0]:
def get_ftp_file_names_economic():
    #get file names
    try:
        import urllib.request as urllib2
    except ImportError:
        import urllib2
    import re
    urlpath =urlopen('ftp://ftp.nass.usda.gov/quickstats')
    string = urlpath.read().decode('utf-8')
    split = string.split('\n')
    file_names = []
    for line in split:
      sub_split = line.split(' ')
      name = sub_split[-1]
      file_names.append(name)
    for name in file_names:
      if "economic" in name:
          print(name)
          return name

def pull_files(file_name):
  file_name = file_name[0:-1]
  ftp_path ='ftp://ftp.nass.usda.gov/quickstats/'+str(file_name)
  return ftp_path

In [0]:
#Get Data from USDA FTP server - this may take a minute
file_name = get_ftp_file_names_economic()
url = pull_files(file_name)
print(url)
req = ur.Request(url) #  gz file on ftp (ensure startswith 'ftp://')
z_f = ur.urlopen(req)
f = GzipFile(fileobj=z_f, mode="r")
df = pd.read_csv(f, sep='\t')

# Select State by changing it too the value below to the states initials
## Note at this point in time NJ is the only state



In [0]:
STATE = "NJ"

In [0]:
  df_economic = df.loc[((df["STATE_ALPHA"] == STATE))]

In [0]:
def add_county_fip(df):
  look_up_dict = {"ATLANTIC":"34001","BERGEN":"34003","BURLINGTON":"34005","CAMDEN":"34007","CAPE MAY":"34009","CUMBERLAND":"34011","ESSEX":"34013","GLOUCESTER":"34015","HUDSON":"34017","HUNTERDON":"34019","MERCER":"34021","MIDDLESEX":"34023","MONMOUTH":"34025","MORRIS":"34027","OCEAN":"34029","PASSAIC":"34031","SALEM":"34033","SOMERSET":"34035","SUSSEX":"34037","UNION":"34039","WARREN":"34041"}
  fip = []
  for ind in df.index:
    county = (df["COUNTY_NAME"][ind])
    code = look_up_dict[county]
    fip.append(str(code))
  df["COUNTY_FIP"]=fip
  return df

def value_convert(df_economic):
  arr = df_economic["VALUE"]
  arr_out = []
  for val in arr:
    string_val = str(val)
    string_val=string_val.replace(",", "")
    if string_val == '(D)' or string_val == '(z)' or string_val == '(Z)':
      string_val = "0"
    arr_out.append(float(string_val))
  df_economic["VALUE"] = arr_out
  return df_economic

def make_widget_year(frame):
  YEAR = widgets.Dropdown(
      options=sorted(frame.YEAR.unique()),
      description="YEAR:",
      disabled=False)
  display(YEAR)
  return YEAR

def make_plot(df_economcic_sub):
  fig = px.choropleth_mapbox(df_economcic_sub, geojson=counties, locations='COUNTY_FIP', color='VALUE',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=6, center = {"lat": 40.5, "lon": -74.871826},
                           opacity=0.8, hover_name="COUNTY_NAME"
                          )
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  #fig = go.FigureWidget()
  #trace = go.Choroplethmapbox(geojson=counties, locations=df_economcic_sub.COUNTY_FIP, z=df_economcic_sub.VALUE,
  #                                  colorscale="Viridis", 
  #                                  marker_opacity=.8, marker_line_width=.5)
  #fig.update_layout(mapbox_style="carto-positron",mapbox_zoom = 6,
  #                mapbox_center = {"lat": 40.5, "lon": -74.871826})
  #
  #fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  #fig.add_trace(trace)
  return fig  

def check(in_put, dataframe):    
  SHORT_DESC_ARR = dataframe.SHORT_DESC.unique()
  if in_put in SHORT_DESC_ARR:
      df_economcic_sub = dataframe[(dataframe["SHORT_DESC"] == (in_put))]
  else:
      df_economcic_sub = dataframe[(dataframe["YEAR"] == int(in_put))]
  arr_year = df_economcic_sub.YEAR.unique()
  arr_year_frames = {}
  if(len(arr_year) > 1):
      for year in arr_year:
          df_economcic_sub_year = df_economcic_sub[(df_economcic_sub["YEAR"] == (year))]
          arr_year_frames[year]=(df_economcic_sub_year)
      widget_year = make_widget_year(df_economcic_sub)
      return arr_year_frames, widget_year
  else:
      cat = str(df_economcic_sub.iloc[0]['SHORT_DESC']) 
      year = str(df_economcic_sub.iloc[0]['YEAR']) 
      string = cat + " " + year
      print(string)
      fig = make_plot(df_economcic_sub)
      fig.show()
      return (0,0)

In [0]:
COUNTY_ARR = ['BERGEN', 'ESSEX', 'HUNTERDON', 'MORRIS', 'PASSAIC',
       'SOMERSET', 'SUSSEX', 'UNION', 'WARREN', 'BURLINGTON', 'MERCER',
       'MIDDLESEX', 'MONMOUTH', 'OCEAN', 'ATLANTIC', 'CAMDEN', 'CAPE MAY',
       'CUMBERLAND', 'GLOUCESTER', 'SALEM', 'HUDSON']
for ind in df_economic.index:
  COUNTY = df_economic["COUNTY_NAME"][ind]
  if COUNTY not in COUNTY_ARR:
    df_economic.at[ind, 'COUNTY_NAME'] = "drop"
df_economic = df_economic[(df_economic["COUNTY_NAME"] != 'drop')]
df_economic=add_county_fip(df_economic)
df_economic=value_convert(df_economic)

# Using the interactive portion
### As I am still working out the kinks, this is a multistep process. Please follow the below steps to see data plotted on a map

1.   Begin by running the below box with the comment **Box 1** - this will present the first interactive drop down to allow you to select the data by description
2.   Next run **Box 2** which will either present the plot if there is only a single year for the data. Otherwise, it will present an additional drop down of year 
3.  Finally, after selecting a year run **Box 3** which will present the output map



In [0]:
#Box 1
SHORT_DESC = widgets.Dropdown(
      options=sorted(df_economic.SHORT_DESC.unique()),
      description='SHORT DESCRIPTION:',
      disabled=False)
display(SHORT_DESC)

In [0]:
#Box 2
#after selecting in above menu, run me - I will either present a map or offer you a list of years
entry_A = SHORT_DESC.value
frames_A,YEAR_A = check(entry_A, df_economic)

In [0]:
#Box 3
#if map did not display above, select a year from the above dropdown and run me
entry = YEAR_A.value
frames,YEAR = check(entry, frames_A[entry])